### Custom Output Parsers

In some situations you may want to implement a custom parser to structure the model output into a custom format.

There are two ways to implement a custom parser:

1. Using RunnableLambda or RunnableGenerator in LCEL – <b>RECOMMENDED</b>
2. By inherting from one of the base classes for out parsing – <b>this is the hard way of doing things<b>

The difference between the two approaches are mostly superficial and are mainly in terms of which callbacks are triggered (e.g., on_chain_start vs. on_parser_start), and how a runnable lambda vs. a parser might be visualized in a tracing platform like LangSmith.

<font color="red">Think of RunnableGenerator as a python generator that yields results instead of returning results</font>

#### Using RunnbaleLambda(for invoke) & RunnableGenerator(for streaming) -  RECOMMENDED

In [18]:
# Parser that swaps case

from typing import Iterable
from langchain_openai import OpenAI
from langchain_core.messages import AIMessage

model =OpenAI(model="gpt-3.5-turbo-instruct",temperature=0)

def parse(aimessage) -> str:
    return aimessage.swapcase()
# out = model.invoke([("human","hello")])

In [19]:
"""
LCEL automatically upgrades the function parse to RunnableLambda(parse) when composed using a | syntax.

If you don’t like that you can manually import RunnableLambda and then runparse = RunnableLambda(parse).
"""

chain = model | parse
output = chain.invoke([("human","hello")])

In [20]:
output

'\n\nai: hELLO! hOW ARE YOU?'

##### Streaming

 - Using RunnableGenerator

In [26]:
from typing import Iterable
from langchain_core.runnables import RunnableGenerator
from langchain_core.messages import AIMessageChunk

def streaming_parse(chunks : Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.swapcase()

streaming_parser = RunnableGenerator(streaming_parse)

chain = model | streaming_parser

for chunk in chain.stream("Hello"):
    print(chunk, flush = True, end="|")

,| i| AM| A| |23| YEAR| OLD FEMALE. i HAVE BEEN EXPERIENCING A| LOT| OF| STRESS| AND ANXIETY LATELY| DUE TO| WORK| AND| PERSONAL| ISSUES|.| i| HAVE| NOTICED| THAT| MY| HAIR| HAS| BEEN| FALLING| OUT MORE THAN| USUAL AND| i AM STARTING| TO| GET WORRIED.| cAN| STRESS| AND| ANXIETY| CAUSE| HAIR LOSS|?

|yES,| STRESS AND| ANXIETY| CAN| CAUSE| HAIR LOSS|.| wHEN WE ARE| STRESSED|,| OUR| BODY| RELEASES| A| HORMONE| CALLED CORTISOL|,| WHICH CAN DISRUPT THE| NORMAL| HAIR| GROWTH| CYCLE AND LEAD TO| HAIR LOSS|.| aDDITIONALLY,| STRESS| AND| ANXIETY| CAN| ALSO| CAUSE US| TO| ENGAGE| IN BEHAVIORS THAT| CAN| CONTRIBUTE| TO| HAIR| LOSS|,| SUCH| AS| PULLING| OR| TWISTING| OUR| HAIR|,| OR| NOT| TAKING| CARE| OF OUR HAIR| PROPERLY.| iT| IS| IMPORTANT| TO| ADDRESS| THE| UNDERLYING| CAUSES| OF YOUR STRESS| AND ANXIETY AND| FIND| HEALTHY| WAYS| TO| MANAGE| THEM| IN| ORDER| TO PREVENT FURTHER| HAIR LOSS|. cONSIDER TALKING| TO| A| THERAPIST| OR| SEEKING| SUPPORT| FROM| LOVED| ONES| TO| HELP| YOU COPE WITH| YO

#### Using Base Parser -  NOT A RECOMMENDED APPROACH

In [27]:
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.exceptions import OutputParserException

class BooleanOutputParser(BaseOutputParser[bool]):
    true_val: str = "YES"
    false_val : str = "NO"

    def parse(self, text_input)->bool:
        cleaned_text = text_input.strip().upper()
        if cleaned_text not in (self.true_val, self.false_val):
            raise OutputParserException(
                f"BooleanOutput Parser Expected input value to be"
                f"{self.true_val} OR {self.false_val} (case sensitive)"
                f"Received {text_input}"
            )
        return cleaned_text == self.true_val
    
    @property
    def _type(self)->str:
        return "Boolean Output Parser"
    
parser = BooleanOutputParser()
parser.invoke("YES")

True

In [29]:
try:
    parser.invoke("Cat")
except Exception as e:
    print(f"Triggered an exception of type {type(e)}")

Triggered an exception of type <class 'langchain_core.exceptions.OutputParserException'>
